In [1]:
import requests
import pandas as pd
import json
from oauth2client.client import OAuth2WebServerFlow
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build

In [2]:
#API key from console.cloud.google.com -> documentation: https://developers.google.com/youtube/v3/docs
API_KEY = "" # Enter your API KEY
#Channel ID for my YouTube Channel
CHANNEL_ID = "" # Enter your CHANNEL_ID

In [3]:
CLIENT_ID = ''
CLIENT_SECRET = ''
SCOPE = 'https://www.googleapis.com/auth/yt-analytics.readonly'
API_SERVICE_NAME = 'YouTube API Analytics'
API_VERSION = 'v2'

In [4]:
def initialize_youtube_analytics():
    #Defining the scopes
    scopes = ["https://www.googleapis.com/auth/yt-analytics.readonly"]

    #Creating the flow using the client_secrets file
    flow = InstalledAppFlow.from_client_secrets_file(
        'client_secret.json', scopes=scopes) #enter your client_secret.json file
    
    #Enter Credentials here
    credentials = flow.run_local_server(port=0)

    #Building the YouTube Analytics service
    return build('youtubeAnalytics', 'v2', credentials=credentials)

In [8]:
def get_video_details(video_id, subscriber_count, youtube_analytics):
    try:
        #Fetching basic video details using YouTube Data API
        data_url = f"https://www.googleapis.com/youtube/v3/videos?id={video_id}&part=snippet,statistics,contentDetails,status&key={API_KEY}"
        data_response = requests.get(data_url).json()

        #Extracting basic details
        snippet = data_response['items'][0]['snippet']
        statistics = data_response['items'][0]['statistics']
        content_details = data_response['items'][0]['contentDetails']
        status = data_response['items'][0]['status']

        #Initializing result dictionary with basic details
        video_details = {
            'video_id': video_id,
            'title': snippet['title'],
            'description': snippet['description'],
            'upload_date': snippet['publishedAt'],
            'duration': content_details['duration'],
            'category_id': snippet['categoryId'],
            'view_count': statistics['viewCount'],
            'like_count': statistics['likeCount'],
            'comment_count': statistics.get('commentCount', 'Not Available'),
            'video_status': status['privacyStatus'],
            'definition': content_details.get('definition', 'Not Available')
        }

  #Fetching additional analytics excluding 'impressions', 'impressionBasedCtr', 'uniqueViewers'
        analytics_response = youtube_analytics.reports().query(
            ids='channel==MINE',
            metrics='averageViewDuration,subscribersGained,subscribersLost',
            dimensions='video',
            filters=f'video=={video_id}',
            startDate='2019-01-01', 
            endDate='2024-01-01'    
        ).execute()

        if analytics_response.get('rows'):
            analytics_data = analytics_response['rows'][0]
            video_details.update({
                'average_view_duration': analytics_data[1],
                'subscribers_gained': analytics_data[2],
                'subscribers_lost': analytics_data[3]
            })
        else:
            video_details.update({
                'subscribers_lost': 'Not Available'
            })

        return video_details

    except requests.exceptions.RequestException as e:
        print(f"Request error occurred: {e}")
        return None
    except KeyError as e:
        print(f"Key error occurred: {e}")
        return None
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return None

In [9]:
#Function to get channel videos and include additional metrics
def get_channel_videos(channel_statistics, youtube_analytics):
    videos = []
    page_token = ""
    while True:
        url = f"https://www.googleapis.com/youtube/v3/search?key={API_KEY}&channelId={CHANNEL_ID}&part=snippet,id&maxResults=50&pageToken={page_token}"
        response = requests.get(url).json()
        for item in response['items']:
            if item['id']['kind'] == "youtube#video":
                video_data = get_video_details(item['id']['videoId'], channel_statistics['subscriber_count'], youtube_analytics)
                videos.append(video_data)
        page_token = response.get('nextPageToken', '')
        if not page_token:
            break
    return pd.DataFrame(videos)

In [10]:
youtube_analytics = initialize_youtube_analytics()
channel_statistics = get_channel_statistics()
save_channel_statistics_to_json(channel_statistics)
video_df = get_channel_videos(channel_statistics, youtube_analytics)

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=110418861456-ljitvjolsts3oski1i1g3kddhe1ecnrh.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A50969%2F&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fyt-analytics.readonly&state=5QwJ75kFKgl3Esstru4frhydJKGA4b&access_type=offline


In [11]:
video_df.head(100)

,video_id,title,description,upload_date,duration,category_id,view_count,like_count,comment_count,video_status,definition,average_view_duration,subscribers_gained,subscribers_lost
0,mrf4zoEucME,types of Rank Pushers in CODM,#codm #shorts,2021-05-28T06:02:46Z,PT59S,20,541263,21352,1749,public,hd,48,958,131
1,W_PEo0cOrK0,Evolution of COD Games on Mobile,#codm #shorts,2021-11-08T09:42:38Z,PT38S,20,13674,707,126,public,hd,32,36,6
2,nofGNDtBuc8,Invisibility Glitch and its Counter (CODM),#codm #codmobile #shorts,2021-03-16T07:29:08Z,PT58S,20,756,55,19,public,hd,37,1,1
3,8URFgkpgoBQ,MOST annoying thing in CODM right now,,2023-02-06T11:06:13Z,PT57S,20,1909,94,29,public,hd,40,5,3
4,XgrRqFOXn8c,and you thought you were having a bad day,#codm #shorts,2021-05-26T06:12:09Z,PT37S,20,28264,2031,175,public,hd,31,59,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,OM6VDSOPhXk,top most saddest death in CODM history,#codm #shorts,2021-04-29T10:51:07Z,PT16S,20,4460,163,15,public,hd,14,4,0
96,p4ZWWTFolVg,when CODM removes the rename card,#codm #shorts,2021-06-15T16:00:49Z,PT45S,20,115977,5903,1270,public,hd,34,471,37
97,KmPDOQEN4jQ,every Free2Play player wants THIS to be back,#codm #shorts\n\nClip Credits: SIGMA CODM,2021-10-06T08:40:19Z,PT35S,20,20744,1167,172,public,hd,29,38,7
98,TPE54pbvlVI,making iFerg moan while getting destroyed in S...,#codm #shorts\n\n@ICECODM new sound effect? 😂,2021-08-02T06:39:44Z,PT51S,20,10460,852,88,public,hd,40,30,5


In [13]:
csv_file_path = 'C:\\Users\\Administrator\\Desktop\\YT_Data_Collected.xlsx'
video_df.to_csv(csv_file_path, index=False)
print(f"DataFrame successfully exported to {csv_file_path}")

DataFrame successfully exported to C:\Users\Administrator\Desktop\YT Analysis\YT_Data_Collected.xlsx
